<a href="https://colab.research.google.com/github/JayThibs/gpt-experiments/blob/main/notebooks/Fine_Tuning_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to GPT

GPT stands for "Generative Pre-Trained Transformer."

* Generative because it is used to generate text.
* Pre-trained because it was trained on a large corpus of unstructured text to make its weights learn things from text like structure, syntax, and general knowledge.
* Transformer because it uses the `decoder` part of the transformer architecture. In other words, you can give it text and it will decode what it needs to output in response (by guessing the next words that follow the input text).

GPT-3 is a massive model. Much too massive to fit in a puny Google Colab GPU and its RAM. Therefore, here we'll use its predecessor, GPT-2, since we can actually fit in on our Colab machine.

GPT models are particular cool because they are able to be applied to many downstream NLP tasks without having to fine-tune the model. Through, few-shot learning, the model can predict what should come next. However, the only current limitation is that the model is limited by its window size. In other words, a model like GPT-J can only fit 2048 tokens as input. That means that in some cases, we might not be able to fit in enough examples to get fantastic results. And when we're in a production environment, it can often be worth it to fine-tune a GPT model to your type of data so that it can perform better.

Models like GPT-3 have been show to show a lot of great results across many different tasks without fine-tuning, even when we compare them to a model like BERT that was specifically fine-tuned on the data. However, it is still often recommended to fine-tune the model to get even better performance. And, in cases like medical data, GPT-3 doesn't necessarily perform well compared to a model like BioBERT.

Perhaps a good rule of thumb is to start by doing creative prompt engineering with your GPT model first to try to get great results, and then you can decide afterwards if you'd like to fine-tune the model for even better accuracy.